In [1]:
# Загрузка объекта из файла
import pickle
with open('/Users/sashamorozov/Downloads/neural_bco_exp_vo_pp_0.33_op_0.33_cp_0.33_generated_routes.pkl', 'rb') as f:
    routes = pickle.load(f)[0].tolist()

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
# 1. Загрузим координаты
coord_file = "/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/datasets/vo_dataset/VOCoords.txt"

# 1. Загрузка координат
with open(coord_file, "r") as f:
    lines = f.readlines()[1:]  # Пропускаем первую строку (кол-во точек)
coords = [list(map(float, line.strip().split())) for line in lines]
df = pd.DataFrame(coords, columns=["x", "y"])

# 2. Создание GeoDataFrame в системе UTM (метры)
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["x"], df["y"]),
    crs="EPSG:32636"  # UTM зона 35N, Санкт-Петербург
)

# # 3. Преобразование в WGS84 (широта/долгота)
gdf = gdf.to_crs("EPSG:4326")
gdf['id'] = gdf.index

In [3]:
import geopandas as gpd
import folium
from shapely.geometry import LineString
import random
import math

def plot_routes_on_map(routes, gdf):
    # Получаем координаты точек
    coords = {row['id']: (row.geometry.y, row.geometry.x) for _, row in gdf.iterrows()}

    # Центр карты
    center_lat = gdf.geometry.y.mean()
    center_lon = gdf.geometry.x.mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

    # Добавляем вершины
    for _, row in gdf.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color='blue',
            fill=True,
            fill_opacity=0.7,
            popup=f"ID: {row['id']}"
        ).add_to(m)

    # Функция генерации случайного цвета
    def random_color():
        return "#{:06x}".format(random.randint(0, 0xFFFFFF))

    # Функция для смещения маршрута
    def offset_route(coords, offset_meters):
        if len(coords) < 2:
            return coords

        offset_coords = []
        for i in range(len(coords)):
            lat, lon = coords[i]

            if i == 0:
                lat2, lon2 = coords[i + 1]
            else:
                lat2, lon2 = coords[i - 1]

            dx = lon2 - lon
            dy = lat2 - lat

            length = math.sqrt(dx**2 + dy**2)
            if length == 0:
                nx, ny = 0, 0
            else:
                nx = -dy / length
                ny = dx / length

            delta_lat = (offset_meters * ny) / 111111
            delta_lon = (offset_meters * nx) / (111111 * math.cos(math.radians(lat)))

            offset_coords.append((lat + delta_lat, lon + delta_lon))

        return offset_coords

    # Добавляем маршруты с сдвигом
    for idx, route in enumerate(routes):
        route_coords = []
        for node_id in route:
            if node_id != -1 and node_id in coords:
                route_coords.append(coords[node_id])

        if len(route_coords) > 1:
            shifted = offset_route(route_coords, offset_meters=idx * 7)
            route_line = folium.PolyLine(
                locations=shifted,
                color=random_color(),
                weight=5,
                opacity=0.9,
                popup=f"Маршрут {idx}"
            )
            fg = folium.FeatureGroup(name=f"Маршрут {idx}")
            fg.add_child(route_line)
            m.add_child(fg)

    folium.LayerControl().add_to(m)
    return m


In [4]:
map_result = plot_routes_on_map(routes, gdf)
map_result